# Data Cleaning

In [1]:
filepath = 'C:/Users/Tim/Desktop/lighthouse/w11,12 - final project/'
data_filepath = filepath+'data/'

import pandas as pd
import numpy as np
import re
import requests
import datetime
from datetime import date, timedelta
from bs4 import BeautifulSoup
import pickle
import copy
import math
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
# ranking_df  
# rank_df_pickle = 'rank_df.pickle'
# rankings = pd.read_pickle(data_filepath+rank_df_pickle)
# with open(data_filepath+rank_df_pickle, 'rb') as f:
#     rankings = pickle.load(f) 
#     rankings = pd.DataFrame(rankings)
rank_df_csv = 'rank_df.csv'
ranking_df = pd.read_csv(data_filepath+rank_df_csv)

# matches_df  
# match_df_pickle = 'match_df.pickle'
# matches = pd.read_pickle(data_filepath+match_df_pickle)
# with open(data_filepath+match_df_pickle, 'rb') as f:
#     matches = pickle.load(f) 
#     matches = pd.DataFrame(matches)

match_df_csv = 'match_df.csv'
match_df = pd.read_csv(data_filepath+match_df_csv)

# team_df  
# team_df_pickle = 'team_df.pickle'
# team_stats = pd.read_pickle(data_filepath+team_df_pickle)
# with open(data_filepath+team_df_pickle, 'rb') as f:
#     team_stats = pickle.load(f) 
#     team_stats = pd.DataFrame(team_stats)

team_df_csv = 'team_df.csv'
team_df = pd.read_csv(data_filepath+team_df_csv)

In [3]:
def info(x):
    n_missing = x.isnull().sum().sort_values(ascending=False)
    p_missing = (x.isnull().sum()/x.isnull().count()).sort_values(ascending=False)
    dtype = x.dtypes
    count = x.count()
    missing_ = pd.concat([n_missing, p_missing, dtype, count],axis=1, keys = [
        'number_missing',
        'percent_missing',
        'type',
        'count'
    ])
    return missing_

In [4]:
ranking_df = ranking_df.drop(columns = 'Unnamed: 0')
match_df = match_df.drop(columns = 'Unnamed: 0')
team_df = team_df.drop(columns = 'Unnamed: 0')

In [5]:
info(ranking_df)

,number_missing,percent_missing,type,count
date,0,0.0,object,55650
rank,0,0.0,object,55650
team,0,0.0,int64,55650
player_names,0,0.0,object,55650


In [6]:
info(match_df)

,number_missing,percent_missing,type,count
player,0,0.0,object,34273
date,0,0.0,object,34273
team,0,0.0,object,34273
team_rounds,0,0.0,int64,34273
opposing_team,0,0.0,object,34273
opposing_team_rounds,0,0.0,int64,34273
map,0,0.0,object,34273
kills,0,0.0,int64,34273
deaths,0,0.0,int64,34273
differential,0,0.0,int64,34273


In [7]:
info(team_df)

,number_missing,percent_missing,type,count
month,0,0.0,object,46245
team,0,0.0,object,46245
p_rounds_won,0,0.0,object,46245
opening_duels,0,0.0,object,46245
multi_kills,0,0.0,float64,46245
team_5v4,0,0.0,object,46245
team_4v5,0,0.0,object,46245
team_traded,0,0.0,object,46245
utility_adr,0,0.0,object,46245
utility_flash,0,0.0,object,46245


# Data Cleaning, Manipulation

In [8]:
ranking_df.head()

,date,rank,team,player_names
0,2015-10-05,fnatic,1,"['pronax', 'olofmeister', 'flusha', 'JW', 'KRI..."
1,2015-10-06,fnatic,1,"['pronax', 'olofmeister', 'flusha', 'JW', 'KRI..."
2,2015-10-07,fnatic,1,"['pronax', 'olofmeister', 'flusha', 'JW', 'KRI..."
3,2015-10-08,fnatic,1,"['pronax', 'olofmeister', 'flusha', 'JW', 'KRI..."
4,2015-10-09,fnatic,1,"['pronax', 'olofmeister', 'flusha', 'JW', 'KRI..."


In [9]:
ranking_df = ranking_df.rename(columns = {
    'rank': 'team',
    'team': 'rank'
})

In [10]:
ranking_df['date'] = pd.to_datetime(ranking_df.date)

In [11]:
# merging team_df into match_df, merge on month and team; change month format
match_df['date'] = pd.to_datetime(match_df.date)
team_df = team_df.rename(columns = {'month':'date'})
team_df['date'] = pd.to_datetime(team_df.date)

In [12]:
match_df.head()

,player,date,team,team_rounds,opposing_team,opposing_team_rounds,map,kills,deaths,differential,rating,avg_impact
0,s1mple,2020-01-03,Natus Vincere,16,G2,2,mrg,14,2,12,1.57,1.34
1,s1mple,2020-01-03,Natus Vincere,16,G2,13,d2,26,19,7,1.35,1.34
2,s1mple,2020-01-03,Natus Vincere,16,G2,4,nuke,19,8,11,1.73,1.34
3,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,nuke,18,11,7,1.52,1.34
4,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,d2,25,11,14,2.09,1.34


In [13]:
team_df.head()

,date,team,p_rounds_won,opening_duels,multi_kills,team_5v4,team_4v5,team_traded,utility_adr,utility_flash
0,2015-10-01,Envy,59.1%,53.7%,0.96,78.7%,36.5%,-,-,-
1,2015-10-02,Envy,59.1%,53.7%,0.96,78.7%,36.5%,-,-,-
2,2015-10-03,Envy,59.1%,53.7%,0.96,78.7%,36.5%,-,-,-
3,2015-10-04,Envy,59.1%,53.7%,0.96,78.7%,36.5%,-,-,-
4,2015-10-05,Envy,59.1%,53.7%,0.96,78.7%,36.5%,-,-,-


In [14]:
df = copy.deepcopy(match_df)
df = df.merge(team_df, on=['date','team'])

df.head()

,player,date,team,team_rounds,opposing_team,opposing_team_rounds,map,kills,deaths,differential,rating,avg_impact,p_rounds_won,opening_duels,multi_kills,team_5v4,team_4v5,team_traded,utility_adr,utility_flash
0,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,nuke,18,11,7,1.52,1.34,55.9%,54.6%,0.87,74.5%,33.6%,19.2%,22.0,0.25
1,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,d2,25,11,14,2.09,1.34,55.9%,54.6%,0.87,74.5%,33.6%,19.2%,22.0,0.25
2,electronic,2020-02-29,Natus Vincere,16,Astralis,5,nuke,22,5,17,1.86,1.21,55.9%,54.6%,0.87,74.5%,33.6%,19.2%,22.0,0.25
3,electronic,2020-02-29,Natus Vincere,16,Astralis,5,d2,15,11,4,1.16,1.21,55.9%,54.6%,0.87,74.5%,33.6%,19.2%,22.0,0.25
4,flamie,2020-02-29,Natus Vincere,16,Astralis,5,nuke,23,10,13,1.79,1.03,55.9%,54.6%,0.87,74.5%,33.6%,19.2%,22.0,0.25


In [15]:
info(df)

,number_missing,percent_missing,type,count
player,0,0.0,object,28297
date,0,0.0,datetime64[ns],28297
utility_adr,0,0.0,object,28297
team_traded,0,0.0,object,28297
team_4v5,0,0.0,object,28297
team_5v4,0,0.0,object,28297
multi_kills,0,0.0,float64,28297
opening_duels,0,0.0,object,28297
p_rounds_won,0,0.0,object,28297
avg_impact,0,0.0,float64,28297


In [16]:
# df['utility_adr'].unique()
# df['utility_flash'].unique()
# df['rating'].unique()
df['team_traded'].unique()

array(['19.2%', '21.6%', '16.6%', '21.4%', '20.7%', '18.2%', '20.0%',
       '21.1%', '22.3%', '19.6%', '18.6%', '23.2%', '21.9%', '18.0%',
       '21.2%', '20.8%', '20.1%', '18.8%', '18.9%', '18.5%', '22.0%',
       '17.5%', '19.9%', '17.8%', '18.1%', '21.8%', '20.2%', '19.5%',
       '22.4%', '22.6%', '15.9%', '19.7%', '17.9%', '15.7%', '18.3%',
       '14.8%', '18.4%', '16.2%', '17.0%', '22.8%', '19.4%', '20.9%',
       '16.9%', '19.3%', '20.5%', '21.3%', '20.3%', '16.5%', '15.1%',
       '20.6%', '22.2%', '-', '19.0%', '20.4%', '19.8%', '21.5%', '17.7%',
       '17.3%', '21.0%', '22.5%', '18.7%', '19.1%', '16.0%', '17.6%',
       '23.5%', '16.7%', '17.2%', '16.3%', '17.4%', '23.8%', '25.2%',
       '23.9%', '24.3%', '22.7%', '22.1%', '24.1%', '23.1%', '23.3%',
       '24.7%', '25.9%', '21.7%', '23.4%', '16.8%', '14.4%', '24.9%',
       '22.9%', '23.6%', '23.0%', '17.1%', '15.6%', '15.2%', '11.8%',
       '25.0%'], dtype=object)

In [17]:
df['rating'] = df['rating'].replace(to_replace='\*',value='',regex=True)
df['rating'] = pd.to_numeric(df['rating'])

In [18]:
# labeling win vs loss
df['win'] = df['team_rounds'] > df['opposing_team_rounds']

In [19]:
le = LabelEncoder()
df['target'] = le.fit_transform(df['win'])
df = df.drop(columns = 'win')

In [20]:
# changing %s to floats

df['p_rounds_won'] = df['p_rounds_won'].replace(to_replace='\%',value='',regex=True)
df['p_rounds_won'] = pd.to_numeric(df['p_rounds_won'])
df['p_rounds_won'] = df['p_rounds_won'] / 100

df['opening_duels'] = df['opening_duels'].replace(to_replace='\%',value='',regex=True)
df['opening_duels'] = pd.to_numeric(df['opening_duels'])
df['opening_duels'] = df['opening_duels'] / 100

df['team_5v4'] = df['team_5v4'].replace(to_replace='\%',value='',regex=True)
df['team_5v4'] = pd.to_numeric(df['team_5v4'])
df['team_5v4'] = df['team_5v4'] / 100

df['team_4v5'] = df['team_4v5'].replace(to_replace='\%',value='',regex=True)
df['team_4v5'] = pd.to_numeric(df['team_4v5'])
df['team_4v5'] = df['team_4v5'] / 100

df['team_traded'] = df['team_traded'].replace(to_replace='\-',value='',regex=True)
df['team_traded'] = df['team_traded'].replace(to_replace='\%',value='',regex=True)
df['team_traded'] = pd.to_numeric(df['team_traded'])
df['team_traded'] = df['team_traded'] / 100

In [21]:
df['utility_adr'] = df['utility_adr'].replace(to_replace='\-',value='',regex=True)
df['utility_adr'] = pd.to_numeric(df['utility_adr'])


df['utility_flash'] = df['utility_flash'].replace(to_replace='\-',value='',regex=True)
df['utility_flash'] = pd.to_numeric(df['utility_flash'])

In [22]:
df['team_traded'] = df['team_traded'].fillna(df.groupby('team')['team_traded'].mean())
df['utility_adr'] = df['utility_adr'].fillna(df.groupby('team')['utility_adr'].mean())
df['utility_flash'] = df['utility_flash'].fillna(df.groupby('team')['utility_flash'].mean())

In [23]:
df['team_traded'] = df['team_traded'].interpolate(method='linear',limit_direction='both')
df['utility_adr'] = df['utility_adr'].interpolate(method='linear',limit_direction='both')
df['utility_flash'] = df['utility_flash'].interpolate(method='linear',limit_direction='both')

# Feature Engineering

Start off with creating features with what we have:
* take ranks from ranking dataframe - can take a team's rank and the opposing team's rank - gives an overall idea for power of a team
* grab stats from kills/deaths/rounds
* * damage would be a better statistic than kills

In [24]:
# rankings per date
def get_rank(opponent, date):
    ranks = ranking_df[ranking_df['date'] == date]
    if len(ranks) == 0:
        return -1
    rows = ranks[ranks['team'] == opponent]
    if(len(rows) == 0):
        return -1
    row = rows.iloc[[0]]
    if len(row) == 0:
        return -1
    return row['rank'].iloc[0]

In [25]:
# ranks per team vs opponent
df['opposing_team_rank'] = -1
df['team_rank'] = -1
for index, row in df.iterrows():
    df.at[index, 'opposing_team_rank'] = get_rank(
        row['opposing_team'], row['date'])
    df.at[index, 'team_rank'] = get_rank(
        row['team'], row['date'])

# stats for each player per game
if df['deaths'] != 0:
    df['kdr'] = df['kills'] / df['deaths']
else:
    df['kdr'] = df['kills'] / (df['deaths'] + 1)
df['kpr'] = df['kills'] / (df['team_rounds'] + df['opposing_team_rounds'])

We want to capture the impactfulness of a player while also capturing their performance in a single feature (we currently have 'impact' for impact and 'rating' for performance). Rating - or a player's overall performance in the game - directly translates to how well a game is going; if all players pop off, then the game is an easy win, regardless of impact. Impact as a statistic really starts to shine when the game is down to the wire. Thus we manipulate impact and incorporate it into rating.

We will normalize impact, square it, and add it to rating, penalizing non-impactful players harder. 

It is important to keep in mind that we have taken average impact statistic over all time rather than impact over time, so this project wouldn't reflect the mental growth of a player.

In [26]:
df['n_impact'] = 0.5*((df['avg_impact'])**2 - (df['avg_impact'].mean())**2)
df['performance'] = df['rating'] + df['n_impact']

Rating can also be padded when playing against worse teams. A good team will roll over a worse team from any factor - preparation, game plan, or simply being better duelists. When such is the case, most of the time, every player's rating is impacted positively, biasing the data. For example, a team going against a team 5 ranks below should theoretically be expected to win every time. Since we have ranking data available, we can create a weight for games with imbalanced teams.

For a particular team: 
* going against a team 2 ranks higher: weighting of 1.5
* going against a team within 2 ranks of itself: sigmoid function between 1 and 1.5
* going against a team 2 ranks below: weighting of 1

In [27]:
df['rank_differential'] = df.apply(lambda x: 31 if x['team_rank'] == -1 or x['opposing_team_rank'] == -1 else x['opposing_team_rank'] - x['team_rank'], axis=1)

# using https://www.desmos.com/calculator to model the weights
def rank_weight(row):
    if row['rank_differential'] < -2:
        return 1.5
    elif row['rank_differential'] > 2:
        return 1
    else:
        return 1.75 - ((1 / (1 + math.exp(-0.5*row['rank_differential']))))

df['rank_weight'] = df.apply(rank_weight, axis=1)

# weighted performance
df['w_performance'] = df['performance'] * df['rank_weight']

# performance residual - accounted for impact and ranking of teams
df['perf_resid'] = df['w_performance'] - df['rating']

Inconsistency is a major factor in a player's performance; they might pop off one game and be completely invisible the next. Mental factors such as the stress from being reverse swept in a series, or tilting from losing a really close game, or even nerves from playing a really strong team are just some of the mental hurdles that professionals must overcome. 

Here we will try to capture a little bit of this variance by lagging our performance residual.

In [28]:
df['perf_resid_lag'] = df.groupby('player')['perf_resid'].shift(1)

# df2 = copy.deepcopy(df)

# df2 = df2.set_index(['date', 'player'])
# df2 = df2.unstack()
# df2 = df2['perf_resid'].shift(1)
# df2 = df2.stack(dropna=False)
# df2 = df2.reset_index().sort_values('player')
# df['perf_resid_lag'] = df2['perf_resid']

df['perf_resid_lag'] = df.groupby('player')['perf_resid_lag'].fillna(method='bfill')

Certain teams perform better on certain maps. This is what the community calls map pool. 

This would be a very useful statistic especially since we have map information per game. However, it is very difficult to capture a team's evolving map pool, especially since it could change over night - teams could start being anti-stratted on a map and take the map off their pool, or a roster change could drastically change the team's dynamic within maps. The residual win% of a map for a team after such changes wouldn't reflect how good they are on that map with that kind of noise.

<!-- # but to get this feature, could do
# weights for maps - some teams have better win rates on certain maps
# groupby team, groupby map - win avg --- for both teams; multiply by importance
# get difference?? or just have the two stats -->

<!-- Due to the coronavirus pandemic that started in late 2019/early 2020, professional CSGO matches were moved off LAN tournaments - where everyone gathered together to play in a stadium - to online matches where teams would play from home.

This had prominent effects on certain players and teams:
* teams had varying ping - moving away from a common spot to play means that players would have to play on servers. 
* * ping is the responsiveness of the game; it is a measure of how long it takes for a packet of data to travel from your computer to a server on the internet and back
* * higher ping for professional players makes a big difference
* * lower tier teams that are more used to playing online saw a huge jump in performance
* teams had to play away from each other
* * sometimes may lose team dynamic as these teams are very used to being together very often
* moved off stadium
* * oftentimes you would see teams performing better on the stage - having an audience gives them motivation 
* * again, lower tier teams that wouldn't normally have big audiences had the stress of having an audience saw a huge jump in performance
  
It was not uncommon to see top tier teams fall apart, or for high profile players to take a long break during this season. -->

The coronavirus pandemic will have no effect on the modelling of this dataset since we have taken LAN only games, before tournament organizers began to cancel LAN events and move everything online. It should be noted, however, that any future predictions from March 2020 onward will be inaccurate due to this large shock to the data. To fix this, we would have to take online matches from HLTV from the past - which biases the data, as some top tier teams perform poorly online - as well as the rest of 2020, and model with corona in mind. 

Theoretically, we could use our models to predict matches in 2021, since teams have since adapted to the corona lifestyle.

In [29]:
# fill nas
info(df)

,number_missing,percent_missing,type,count
player,0,0.0,object,28297
date,0,0.0,datetime64[ns],28297
perf_resid,0,0.0,float64,28297
w_performance,0,0.0,float64,28297
rank_weight,0,0.0,float64,28297
rank_differential,0,0.0,int64,28297
performance,0,0.0,float64,28297
n_impact,0,0.0,float64,28297
kpr,0,0.0,float64,28297
kdr,0,0.0,float64,28297


In [30]:
df.to_csv(data_filepath + 'final_df.csv')

In [32]:
df.head()

,player,date,team,team_rounds,opposing_team,opposing_team_rounds,map,kills,deaths,differential,...,team_rank,kdr,kpr,n_impact,performance,rank_differential,rank_weight,w_performance,perf_resid,perf_resid_lag
0,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,nuke,18,11,7,...,6,1.636364,0.857143,0.330758,1.850758,-5,1.5,2.776137,1.256137,1.256137
1,s1mple,2020-02-29,Natus Vincere,16,Astralis,5,d2,25,11,14,...,6,2.272727,1.190476,0.330758,2.420758,-5,1.5,3.631137,1.541137,1.256137
2,electronic,2020-02-29,Natus Vincere,16,Astralis,5,nuke,22,5,17,...,6,4.400000,1.047619,0.165008,2.025008,-5,1.5,3.037512,1.177512,1.177512
3,electronic,2020-02-29,Natus Vincere,16,Astralis,5,d2,15,11,4,...,6,1.363636,0.714286,0.165008,1.325008,-5,1.5,1.987512,0.827512,1.177512
4,flamie,2020-02-29,Natus Vincere,16,Astralis,5,nuke,23,10,13,...,6,2.300000,1.095238,-0.036592,1.753408,-5,1.5,2.630112,0.840112,0.840112
